Install the dependencies we are going to use. JuMP is a Julia mathematical optimization front end and problem builder, CSV parses .csv files, and GLPK is a free open source solver. (add these at the julia repl) with <;add JuMP>

In [4]:
using JuMP
using CSV
using GLPK

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1242


First we will parse the problem specification.

In [57]:
problem = CSV.read("problem.csv"; delim="\t", header=false)
# problem = CSV.read("fixedSudoko16.csv",header=false)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,7,0,0,0,0,0,0,0,6
2,0,0,5,0,0,0,2,0,0
3,0,3,0,4,0,7,0,9,0
4,0,0,2,1,0,8,5,0,0
5,3,0,0,7,0,9,0,0,1
6,0,0,9,2,0,4,3,0,0
7,0,8,0,9,0,1,0,5,0
8,0,0,3,0,0,0,7,0,0
9,1,0,0,0,0,0,0,0,8


Build the problem into a mathematical expression. Note that all data is stored as binary data.

In [58]:
model = Model(
    with_optimizer(GLPK.Optimizer, tm_lim = 60000, msg_lev = GLPK.OFF)
)
numberRows = size(problem, 1)
cellSize = Int(sqrt(numberRows))

@variable(model, entries[1:numberRows,1:numberRows,1:numberRows], Bin)
#only one number in each cell
foreach(1:numberRows) do idxX
    foreach(1:numberRows) do idxY
        @constraint(model, sum(entries[idxX,idxY,:]) == 1)
    end
end
# each row has one of each entry
foreach(1:numberRows) do idxX
    foreach(1:numberRows) do idxEntry 
        @constraint(model, sum(entries[idxX, :, idxEntry]) == 1)
    end
end
# each column has one of each entry
foreach(1:numberRows) do idxY
    foreach(1:numberRows) do idxEntry 
        @constraint(model, sum(entries[:, idxY, idxEntry]) == 1)
    end
end
# each block has one of each entry
foreach(1:cellSize) do blockXidx
    foreach(1:cellSize) do blockYidx
        foreach(1:numberRows) do idxEntry
            rangeX = (blockXidx-1)*cellSize+1:(blockXidx)*cellSize
            rangeY = (blockYidx-1)*cellSize+1:(blockYidx)*cellSize
            @constraint(model, sum(entries[rangeX,rangeY,idxEntry]) == 1)
        end
    end
end
# set the values from the given problem
foreach(1:numberRows) do idxX
    foreach(1:numberRows) do idxY
        foreach(1:numberRows) do idxEntry
            if problem[idxX, idxY] == idxEntry
                @constraint(model, entries[idxX,idxY,idxEntry] == 1)
            end
        end
    end
end

Solve the problem.

In [59]:
optimize!(model)

In [60]:
entryData = round.(value.(entries))
result = [findfirst(val -> val == 1, entryData[idxX,idxY,:]) for idxX in 1:numberRows, idxY in 1:numberRows ]

9×9 Array{Int64,2}:
 7  2  4  8  9  5  1  3  6
 8  9  5  6  1  3  2  7  4
 6  3  1  4  2  7  8  9  5
 4  7  2  1  3  8  5  6  9
 3  6  8  7  5  9  4  2  1
 5  1  9  2  6  4  3  8  7
 2  8  7  9  4  1  6  5  3
 9  4  3  5  8  6  7  1  2
 1  5  6  3  7  2  9  4  8